In [1]:
!pip install faiss-gpu
!pip install sentence-transformers

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import numpy as np
import faiss
import requests
from io import StringIO
import pandas as pd

# Dataset Initialization

In [3]:
res = requests.get('https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/sick2014/SICK_train.txt')

text = res.text
text[:100]

'pair_ID\tsentence_A\tsentence_B\trelatedness_score\tentailment_judgment\n1\tA group of kids is playing in '

We need this in a dataframe, which we build from the `text` string like so:

In [4]:
data = pd.read_csv(StringIO(text), sep='\t')
data.head()

,pair_ID,sentence_A,sentence_B,relatedness_score,entailment_judgment
0,1,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,4.5,NEUTRAL
1,2,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,3.2,NEUTRAL
2,3,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,4.7,ENTAILMENT
3,5,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,3.4,NEUTRAL
4,9,The young boys are playing outdoors and the ma...,A group of kids is playing in a yard and an ol...,3.7,NEUTRAL


We will take all samples from `sentence_A` and build sentence embeddings for each - which we can then store in FAISS.

In [5]:
sentences = data['sentence_A'].tolist()
sentences[:5]

['A group of kids is playing in a yard and an old man is standing in the background',
 'A group of children is playing in the house and there is no man standing in the background',
 'The young boys are playing outdoors and the man is smiling nearby',
 'The kids are playing outdoors near a man with a smile',
 'The young boys are playing outdoors and the man is smiling nearby']

And we will pull in the `sentence_B` column too, giving us ~4.5K *unique* sentences.

In [6]:
sentence_b = data['sentence_B'].tolist()
sentences.extend(sentence_b)
len(set(sentences))

4802

This isn't a particularly large number, so let's pull in a few more similar datasets.

In [7]:
urls = [
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/MSRpar.train.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/MSRpar.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2013/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2014/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2014/images.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2015/images.test.tsv'
]

In [8]:
for url in urls:
    res = requests.get(url)
    # extract to dataframe
    data = pd.read_csv(StringIO(res.text), sep='\t', header=None, error_bad_lines=False)
    # add to columns 1 and 2 to sentences list
    sentences.extend(data[1].tolist())
    sentences.extend(data[2].tolist())

<ipython-input-8-5a6e39e4bd88>:4: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv(StringIO(res.text), sep='\t', header=None, error_bad_lines=False)
b'Skipping line 191: expected 3 fields, saw 4\nSkipping line 206: expected 3 fields, saw 4\nSkipping line 295: expected 3 fields, saw 4\nSkipping line 695: expected 3 fields, saw 4\nSkipping line 699: expected 3 fields, saw 4\n'
<ipython-input-8-5a6e39e4bd88>:4: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv(StringIO(res.text), sep='\t', header=None, error_bad_lines=False)
b'Skipping line 104: expected 3 fields, saw 4\nSkipping line 181: expected 3 fields, saw 4\nSkipping line 317: expected 3 fields, saw 4\nSkipping line 412: expected 3 fields, saw 5\nSkipping line 508: expected 3 fields, saw 4\n'
<ipython-input-8-5a6e3

In [9]:
len(set(sentences))

14505

Before converting to our sentence embeddings, we will save to text file as backup.

In [10]:
# remove duplicates and NaN
sentences = [
    sentence.replace('\n', '') for sentence in list(set(sentences)) if type(sentence) is str
    ]

In [11]:
with open('sentences.txt', 'w') as fp:
    fp.write('\n'.join(sentences))

Now we have 14.5K *unique* sentences, a much better size. We'll go ahead and build the sentence embeddings (this can take some time, feel free to download the embeddings from [here]()). TODO add link

In [12]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('bert-base-nli-mean-tokens')

sentence_embeddings = model.encode(sentences)
sentence_embeddings.shape

(14504, 768)

We can save/load from file in the case of needing to reload the notebook for any reason later.

In [13]:
sentence_embeddings.shape[0]

14504

In [15]:
with open(f'./sim_sentences/embeddings_X.npy', 'wb') as fp:
    np.save(fp, sentence_embeddings[0:256])

In [16]:
# saving data
split = 256
file_count = 0
for i in range(0, sentence_embeddings.shape[0], split):
    end = i + split
    if end > sentence_embeddings.shape[0] + 1:
        end = sentence_embeddings.shape[0] + 1
    file_count = '0' + str(file_count) if file_count < 0 else str(file_count)
    with open(f'./sim_sentences/embeddings_{file_count}.npy', 'wb') as fp:
        np.save(fp, sentence_embeddings[i:end, :])
    print(f"embeddings_{file_count}.npy | {i} -> {end}")
    file_count = int(file_count) + 1

embeddings_0.npy | 0 -> 256
embeddings_1.npy | 256 -> 512
embeddings_2.npy | 512 -> 768
embeddings_3.npy | 768 -> 1024
embeddings_4.npy | 1024 -> 1280
embeddings_5.npy | 1280 -> 1536
embeddings_6.npy | 1536 -> 1792
embeddings_7.npy | 1792 -> 2048
embeddings_8.npy | 2048 -> 2304
embeddings_9.npy | 2304 -> 2560
embeddings_10.npy | 2560 -> 2816
embeddings_11.npy | 2816 -> 3072
embeddings_12.npy | 3072 -> 3328
embeddings_13.npy | 3328 -> 3584
embeddings_14.npy | 3584 -> 3840
embeddings_15.npy | 3840 -> 4096
embeddings_16.npy | 4096 -> 4352
embeddings_17.npy | 4352 -> 4608
embeddings_18.npy | 4608 -> 4864
embeddings_19.npy | 4864 -> 5120
embeddings_20.npy | 5120 -> 5376
embeddings_21.npy | 5376 -> 5632
embeddings_22.npy | 5632 -> 5888
embeddings_23.npy | 5888 -> 6144
embeddings_24.npy | 6144 -> 6400
embeddings_25.npy | 6400 -> 6656
embeddings_26.npy | 6656 -> 6912
embeddings_27.npy | 6912 -> 7168
embeddings_28.npy | 7168 -> 7424
embeddings_29.npy | 7424 -> 7680
embeddings_30.npy | 7680 -> 7

We setup our FAISS database dimensionality (number of dimensions per vector) based on these vectors.

In [17]:
d = sentence_embeddings.shape[1]
d

768

# Flat L2 Index

We initialize the flat L2 distance index `IndexFlatL2`, all we need is the specify the vector dimensionality - which in this case is `d == 768` (to align with the sentence-BERT model output embeddings of size `768`).

In [18]:
index = faiss.IndexFlatL2(d)

Often, we will use indexes that require us to `train` them on our data before being used (if we are grouping or transforming the data in any way). `IndexFlatL2` however, is a simple operation and only requires that we calculate distances between vectors when we introduce our query vector `xq` during search. So, in this case, no training is required - which we can confirm by checking the `is_trained` attribute.

In [19]:
index.is_trained

True

Okay so once we're happy that our index is prepared, we then add new vectors using the `add` method.

In [20]:
index.add(sentence_embeddings)

In [21]:
index.ntotal

14504

Then search given a query `xq` and number of nearest neigbors to return `k`.

In [22]:
k = 4
xq = model.encode(["Someone sprints with a football"])

In [23]:
%%time
D, I = index.search(xq, k)  # search
print(I)  # k-nearest neigbors of the query vector | nprobe == 1: 6495 26392 61709 49932 | nprobe == 10: 36245  6495 57489  8705

[[ 6734 10070  3909  6242]]
CPU times: user 28.3 ms, sys: 0 ns, total: 28.3 ms
Wall time: 26 ms


Here we're returning indices `7460`, `10940`, `3781`, and `5747`, which returns:

In [24]:
[f'{i}: {sentences[i]}' for i in I[0]]

['6734: A group of football players is running in the field',
 '10070: A group of people playing football is running in the field',
 '3909: Two groups of people are playing football',
 '6242: A person playing football is running past an official carrying a football']

In [25]:
sentences[7460]

'the floor of a building that is at or nearest to the level of the ground around the building.'

Clearly we have some good matches, everything returned includes people running with a football, or on the context of a football match. Now, if we'd rather extract the numerical vectors from FAISS, we can do that too.

In [26]:
vecs = np.zeros((k, d))
for i, val in enumerate(I[0].tolist()):
    vecs[i, :] = index.reconstruct(val)

In [27]:
vecs.shape

(4, 768)

In [28]:
vecs[0][:100]

array([ 0.01627047,  0.22325903, -0.15037391, -0.30747262, -0.27122435,
       -0.10593151, -0.06460945,  0.04738196, -0.73349029, -0.37657699,
       -0.76762801,  0.16902876,  0.53107649,  0.51176685,  1.14415872,
       -0.08562902, -0.67240065, -0.96637088,  0.02545451, -0.21559811,
       -1.25656569, -0.82982177, -0.0982501 , -0.21850847,  0.50610214,
        0.10527944,  0.50396889,  0.65242976, -1.39458692,  0.65847439,
       -0.21525328, -0.22487469,  0.81818336,  0.08464261, -0.76141691,
       -0.28928286, -0.09825835, -0.7304619 ,  0.07855791, -0.84354621,
       -0.592421  ,  0.77471334, -1.20920527, -0.22757959, -1.30733597,
       -0.23081471, -1.31322515,  0.01629055, -0.97285479,  0.19308172,
        0.47424567,  1.18920863, -1.96741331, -0.70061129, -0.29638705,
        0.60533714,  0.62407428, -0.70340401, -0.86754197,  0.17673175,
       -0.19170511, -0.02951994,  0.22623582, -0.16695456, -0.80402523,
       -0.45918918,  0.69675475, -0.24928194, -1.01478684, -0.92

---

# Adding Partitioning to the Index

FAISS allows us to add an additional step to optimize our search efficiency using a variety of different methods. A popular approach is to partition the index into *[Voronoi cells](https://en.wikipedia.org/wiki/Voronoi_diagram)*. Using this method we would take our query vector `xq`, identify the *cell* it belongs to, and then use our `IndexFlatL2` to search between the query vector `xq` and all indexed vectors belonging to that *cell*. We can also include vectors from other nearby cells too.

We initialize our new partitioned index by first adding our previous `IndexFlatL2` operation as a quantization step (another step in the search process), and feeding this into the new `IndexIVFFlat` operation like so:

In [29]:
nlist = 50
quantizer = faiss.IndexFlatL2(d)
index = faiss.IndexIVFFlat(quantizer, d, nlist)

Here we've added a new parameter `nlist`. We use `nlist` to define how many partitions we'd like our index to have. 

When we built the previous, `IndexFlatL2`-only index, we noted that no training was required as no grouping/transformation was required to build that index. Now that we've added partitioning using `IndexIVFFlat`, this is no longer the case. Let's take a look at the `is_trained` attribute.

In [30]:
index.is_trained

False

So, what we need to do now is `train` our index on our data, which we do *before* adding any data to the index (otherwise the index cannot know how to group each vector).

In [31]:
index.train(sentence_embeddings)
index.is_trained

True

Now our index is trained, we add our data just as we did before.

In [32]:
index.add(sentence_embeddings)
index.ntotal

14504

Let's search again using the same indexed sentence embeddings and the same query `xq`.

In [33]:
%%time
D, I = index.search(xq, k)  # search
print(I)

[[ 6734 10070 11425  3688]]
CPU times: user 3.83 ms, sys: 669 µs, total: 4.5 ms
Wall time: 2.61 ms


We can increase the number of nearby cells to search too with `nprobe`.

In [34]:
index.nprobe = 10

In [35]:
%%time
D, I = index.search(xq, k)  # search
print(I)

[[ 6734 10070  3909  6242]]
CPU times: user 11.4 ms, sys: 0 ns, total: 11.4 ms
Wall time: 9.6 ms


Increasing the number of `nprobe` will improve the accuracy of our search, but cost time. Our earlier `IndexFlatL2`-only search was *exhaustive* (it compared every single vector) and so it identified the closest matches with a perfect accuracy. The smaller our `nprobe` value, the smaller scope that we search. We received perfect results (that matched our previous `IndexFlatL2`-only results - `7460`, `10940`, `3781`, `5747`), however, if we found that we were not getting closely matching results, we could simply bump `nprobe` up further - improving accuracy, but increasing time-taken too.

It's worth noting that the time taken can change with each run too, if we rerun the above block, we usually get a different time:

In [36]:
%%time
D, I = index.search(xq, k)
print(I)

[[ 6734 10070  3909  6242]]
CPU times: user 6.44 ms, sys: 3.59 ms, total: 10 ms
Wall time: 8.22 ms


For IVF (and IMI) indexes, before attempting to use the `reconstruct` method, we need to call the `make_direct_map` method - otherwise we will return a `RunetimeError`.

In [37]:
index.reconstruct(7460)

RuntimeError: Error in faiss::DirectMap::idx_t faiss::DirectMap::get(faiss::DirectMap::idx_t) const at /project/faiss/faiss/invlists/DirectMap.cpp:81: direct map not initialized

With `make_direct_map`:

In [38]:
index.make_direct_map()

In [39]:
index.reconstruct(7460)[:100]

array([ 5.95892191e-01, -8.37487876e-02,  6.47573709e-01,  5.85240781e-01,
        6.61498070e-01, -1.41456053e-01,  8.46518278e-01,  6.14825249e-01,
       -2.89108872e-01, -1.03891039e+00, -3.23937535e-01, -6.45343423e-01,
        3.42787296e-01,  7.37026811e-01,  8.50309432e-01, -9.36678886e-01,
       -3.86863261e-01, -1.11071579e-01,  7.36397326e-01, -1.45938501e-01,
       -8.63656327e-02,  1.58017486e-01,  3.57938170e-01, -4.57567245e-01,
       -7.32586265e-01, -6.41742170e-01,  7.90953040e-02,  2.56033123e-01,
       -3.88635993e-01,  3.08782179e-02, -1.05497621e-01, -8.48549426e-01,
        5.51437140e-01, -1.00283253e+00,  8.12477291e-01,  5.55458665e-01,
       -7.13642359e-01,  4.71150845e-01,  1.05705492e-01, -6.06462538e-01,
       -3.37060481e-01,  8.93455744e-02,  7.75567651e-01, -1.04590684e-01,
       -9.65191603e-01, -2.26941764e-01,  8.50935519e-01,  3.29864591e-01,
       -2.73874998e-01,  6.12350762e-01, -7.24009871e-02,  3.57083410e-01,
        4.03494358e-01,  

We've now significantly reduced the search time, what can we do next?

# Quantization

Well, when storing these vectors we're storing the full (eg `Flat`) vector. Now in very big datasets this can quickly become a problem. Typically, we look at big datasets, and when working with large dataset we will find that storing the full vectors consumes too much space.

Fortunately, FAISS comes with the ability to *compress* our vectors using transformations based on *Product Quantization* (PQ). But, what is PQ? Well, we can view it as an additional approximation step similar to our use of **IVF**, which allowed us to *approximate* by reducing the scope of our search. PQ is slightly different however, and approximates the distance (or similarity) calculation instead.

[PQ explanation TODO REMOVE](https://mccormickml.com/2017/10/13/product-quantizer-tutorial-part-1/)

PQ achieves this approximated distance operation by compressing the vectors themselves. This consists of a few steps.

1. We split the every original vector into several subvectors.

2. For each set of subvectors, we perform a clustering operation, creating many centroids for each subvector set.

3. In our vector of subvectors, we replace each subvector with the ID of it's nearest centroid.

In [40]:
m = 8  # number of centroid IDs in final compressed vectors
bits = 8 # number of bits in each centroid

quantizer = faiss.IndexFlatL2(d)  # we keep the same L2 distance flat index
index = faiss.IndexIVFPQ(quantizer, d, nlist, m, bits) 

Again, we'll need to `train` the index.

In [41]:
index.is_trained

False

In [42]:
index.train(sentence_embeddings)

And `add` our vectors.

In [43]:
index.add(sentence_embeddings)

Let's compare it to our previous index *without* PQ, and an `nprobe` value of `10`.

In [44]:
index.nprobe = 10

In [45]:
%%time
D, I = index.search(xq, k)
print(I)

[[1052 1623 2405 2231]]
CPU times: user 3.43 ms, sys: 207 µs, total: 3.64 ms
Wall time: 1.95 ms


Through adding PQ we've reduced our search time from ~7.5ms to ~5ms, a small difference on a dataset of this size, but when scaled to larger datasets this can make a huge difference.

Now, we should also notice the slightly different results being returned. Beforehand with our exhaustive L2 search we were returning `7460`, `10940`, `3781`, and `5747`. Now, we see a slightly different order to our results - and two different vectors, `5013` and `5370`.

Each of our speed optimization operations, `IVF` and `PQ`, come at the cost of accuracy. Now, if we print out these results we will nonetheless find that each item is still a relevant match:

In [46]:
[f'{i}: {sentences[i]}' for i in I[0]]

['1052: Two football players bang into each other during a football game.',
 '1623: Two boys playing a game of flag football.',
 '2405: A football player in a red and white uniform is being lifted by two teammates',
 '2231: Two football players are scrambling for the ball on the court']

So although we might not get the *perfect* result, we still get close - and thanks to the approximation, we get a significant speed boost